In [1]:
from tqdm import tqdm
import pandas as pd

In [2]:
# !pip install fastparquet
# !pip install pyarrow

In [3]:
# train упорядочен хронологически
train = pd.read_parquet('train.parquet.gzip')
train.dtypes

user_id      int32
item_id      int32
timespent     int8
reaction      int8
dtype: object

In [26]:
train[['timespent', 'reaction']].describe()

,timespent,reaction
count,1.444400e+08,1.444400e+08
mean,5.446560e-01,8.683487e-03
std,1.690750e+00,1.002283e-01
min,0.000000e+00,-1.000000e+00
25%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00
max,6.000000e+01,1.000000e+00


In [ ]:
pip p

In [27]:
8.683487e-03

0.008683487

In [4]:
# timespent: время залипания юзера на айтем в минутах (от 0 до 60)
# reaction: (1) - лайк, (-1) - дизлайк
train

,user_id,item_id,timespent,reaction
0,707536,67950,0,0
1,707536,151002,0,0
2,707536,134736,0,0
3,707536,196151,0,0
4,707536,94182,0,0
...,...,...,...,...
144440010,849764,80910,0,0
144440011,993316,132328,0,0
144440012,993316,186701,0,0
144440013,666981,81857,0,0


In [6]:
# в items_meta для каждого item_id его автор и эмбеддинг содержания
items_meta = pd.read_parquet(f'items_meta.parquet.gzip')
items_meta

,item_id,source_id,embeddings
0,0,7340,"[0.10458118, 0.047880154, 0.030944156, -0.0351..."
1,1,6284,"[0.035625108, -0.039264094, -0.03310334, -0.04..."
2,2,12766,"[0.08418761, 0.006732465, -0.0037112322, -0.02..."
3,3,14734,"[0.049901545, 0.039079394, -0.03890682, -0.053..."
4,4,22557,"[0.09303163, 0.023448057, 0.0029488814, -0.017..."
...,...,...,...
227601,227601,19043,"[0.06742832, -0.08209568, -0.04407321, 0.00838..."
227602,227602,9384,"[0.07055114, -0.007334651, -0.0032477665, 0.00..."
227603,227603,24152,"[0.13771634, 0.023559753, 0.012204557, -0.0361..."
227604,227604,20249,"[0.04954276, -0.00674311, -0.040121585, -0.024..."


In [7]:
# candidates содержит item_id свежих кандидатов из которых нужно будет предсказать на тесте
candidates_df = pd.read_parquet(f'fresh_candidates.parquet.gzip')
candidates_df

,item_id
0,0
1,2
2,5
3,6
4,7
...,...
99995,227588
99996,227591
99997,227602
99998,227603


In [8]:
test = pd.read_parquet('test.parquet.gzip')
test

,user_id
0,7
1,8
2,9
3,11
4,18
...,...
199995,1000160
199996,1000165
199997,1000166
199998,1000168


In [9]:
class Baseline:
    def __init__(self, n_candidates):
        self.n_candidates = n_candidates
    
    def fit(self, interactions, candidates):
        popularity = interactions.groupby('item_id')['timespent'].mean().values
        candidates_popularity = popularity[candidates]
        most_popular_idxs = (-candidates_popularity).argsort()[:self.n_candidates]
        self.impersonal_prediction = candidates[most_popular_idxs]
        
    def predict(self, user_id):
        return self.impersonal_prediction

In [10]:
baseline = Baseline(20)
baseline.fit(train, candidates_df.item_id.values)

In [37]:
from sklearn.metrics import ndcg_score

In [32]:
predictions = []
for user_id in tqdm(test.user_id):
    predictions.append(baseline.predict(user_id))

100%|██████████| 200000/200000 [00:00<00:00, 1746404.18it/s]


In [33]:
test['predictions'] = predictions
test

,user_id,predictions
0,7,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
1,8,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
2,9,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
3,11,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
4,18,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
...,...,...
199995,1000160,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
199996,1000165,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
199997,1000166,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."
199998,1000168,"[4628, 103927, 146586, 18584, 75560, 44269, 58..."


In [36]:
test.to_parquet('sample_submission.parquet.gzip', compression='gzip', engine='pyarrow')